# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [25]:
# autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [27]:
# data_dir = '../base_and_generated'

batch_size = 64
epochs = 8
workers = 0 if os.name == 'nt' else 4 # 8

#### Determine if an nvidia GPU is available

In [28]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [29]:
# mtcnn = MTCNN(
#     image_size=160, margin=0, min_face_size=20,
#     thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
#     device=device
# )

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [30]:
# dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
# dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
# dataset.idx_to_class

In [31]:
# dataset.samples = [
#     (p, p.replace(data_dir, data_dir + '_cropped'))
#         for p, _ in dataset.samples
# ]

# # loader = DataLoader(
# #     dataset,
# #     num_workers=workers,
# #     batch_size=batch_size,
# #     collate_fn=training.collate_pil
# # )
# loader = DataLoader(dataset, collate_fn=training.collate_pil, num_workers=workers)

# # x: image object
# # y: save path of corresponding object
# for i, (x, y) in enumerate(loader):
#     mtcnn(x, save_path=y)
#     print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# # Remove mtcnn to reduce GPU memory usage
# del mtcnn

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [32]:
# set random seed for reproducibility
np.random.seed(0)
torch.cuda.empty_cache()
resnet = InceptionResnetV1(
    classify=True,
    pretrained='casia-webface',
    num_classes=2
).to(device)

#### Define optimizer, scheduler, dataset, and dataloader

In [33]:
base_lr = 0.001
params = [
    {'params': [param for name, param in resnet.named_parameters() if name in ['logits.weight', 'logits.bias']], 'lr': base_lr},
    {'params': [param for name, param in resnet.named_parameters() if name not in ['logits.weight', 'logits.bias']], 'lr': base_lr * 0.01}
]
optimizer = optim.Adam(params)
scheduler = MultiStepLR(optimizer, [5, 10])

# add random rotation and flip to data augmentation
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(degrees=(-10, 10)),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(10),
    fixed_image_standardization
])
train_data_dir = '../data/base_and_final_cropped'
val_data_dir = '../data/test_images_cropped'
train_dataset = datasets.ImageFolder(train_data_dir, transform=trans)
val_dataset = datasets.ImageFolder(val_data_dir, transform=trans)
train_loader = DataLoader(
    train_dataset,
    num_workers=workers,
    batch_size=batch_size,
    shuffle=True,
)
val_loader = DataLoader(
    val_dataset,
    num_workers=workers,
    batch_size=batch_size,
    shuffle=True,
)

# # freeze all layers except the final classification layer
# for name, param in resnet.named_parameters():
#     if name not in ['logits.weight', 'logits.bias']:
#         param.requires_grad = False

        


#### Define loss and evaluation functions

In [34]:
import torch
from torchmetrics import F1Score, Accuracy
from torchmetrics.classification import BinaryF1Score

from torchmetrics.functional.classification import binary_f1_score


loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy,
    #'tl_acc': Accuracy(task='binary').to(device),
    'fscore': BinaryF1Score().to(device)# F1Score(task="multiclass", average=None, num_classes=2).to(device)
}

#### Train model

In [35]:
# writer = SummaryWriter()
# writer.iteration, writer.interval = 0, 1

print('\n\nInitial')
print('-' * 10)
# resnet.eval()
# training.pass_epoch(
#     resnet, loss_fn, val_loader,
#     batch_metrics=metrics, show_running=True, device=device,
#     writer=writer
# )

epochs = 30
for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=None
    )

    if epoch % 2 == 0:
        resnet.eval()
        training.pass_epoch(
            resnet, loss_fn, val_loader,
            batch_metrics=metrics, show_running=True, device=device,
            writer=None
        )
    #writer.iteration += 1  # Increment writer.iteration after each epoch

# writer.close()



Initial
----------

Epoch 1/30
----------


Train |     2/2    | loss:    0.4792 | fps:  144.9581 | acc:    0.8032 | fscore:    0.8231   
Valid |    23/23   | loss:    0.4545 | fps: 1442.7217 | acc:    0.8367 | fscore:    0.6744   

Epoch 2/30
----------
Train |     2/2    | loss:    0.2392 | fps:  150.9082 | acc:    0.9688 | fscore:    0.9722   

Epoch 3/30
----------
Train |     2/2    | loss:    0.1194 | fps:  151.2059 | acc:    1.0000 | fscore:    1.0000   
Valid |    23/23   | loss:    0.3128 | fps: 1446.2366 | acc:    0.8906 | fscore:    0.7642   

Epoch 4/30
----------
Train |     2/2    | loss:    0.0583 | fps:  152.6493 | acc:    1.0000 | fscore:    1.0000   

Epoch 5/30
----------
Train |     2/2    | loss:    0.0445 | fps:  150.2276 | acc:    1.0000 | fscore:    1.0000   
Valid |    23/23   | loss:    0.2216 | fps: 1460.9882 | acc:    0.9305 | fscore:    0.8246   

Epoch 6/30
----------
Train |     2/2    | loss:    0.0248 | fps:  152.9141 | acc:    1.0000 | fscore:    1.0000   

Epoch 7/30
----------
Train |     2/2 

In [36]:
# import matplotlib.pyplot as plt
# import csv

# # Read data from the CSV file
# epochs = []
# accuracies = []

# with open('results.csv', 'r') as file:
#     csv_reader = csv.DictReader(file)
#     for row in csv_reader:
#         epochs.append(int(row['epoch']))
#         accuracies.append(float(row['accuracy']))

# # Create the plot
# plt.figure(figsize=(8, 6))
# plt.plot(epochs, accuracies, marker='o')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.title('Accuracy vs. Epoch for the model trained on generated images')
# plt.grid(True)
# plt.show()